# **Data Pre-Processing Pipeline**
## **Lý thuyết (Theory)** 

### **Mục tiêu tổng thể**:  
Chuyển đổi dữ liệu thô crawl về nhà/phòng cho thuê (nhiều file CSV) thành dữ liệu sạch, có cấu trúc, sẵn sàng cho EDA và mô hình hóa.

### **Các bước chính**:
1. Gộp dữ liệu
2. Chuẩn hóa giá tiền & diện tích
3. Trích xuất thông tin địa chỉ chi tiết
4. Trích xuất & mã hóa tiện ích từ mô tả
5. Chuẩn hóa ngày đăng
6. Lọc cột & lưu kết quả

<!-- # **Data Pre-Processing**
## **Lý thuyết (Theory)** -->


<!-- ### **Tổng quan luồng xử lý dữ liệu bất động sản (Data Pre-processing Pipeline)**

**Mục tiêu chính**  
Chuẩn bị dữ liệu thô (raw) từ các file crawl về nhà/phòng cho thuê → thành dữ liệu sạch, có cấu trúc, sẵn sàng cho EDA và mô hình hóa (dự đoán giá, phân cụm khu vực, phân loại tiện ích, v.v.). -->

### **Nguồn dữ liệu ban đầu**  
- Nhiều file CSV dạng `PageXtoY.csv` (crawl từ các trang bất động sản)  
- Các cột chính thường gặp: title, price, area, address/location, thongtinmota, ngaydang, url, ...

### **Luồng xử lý tổng thể (End-to-End) – dạng text**

1. **Thu thập & gộp dữ liệu**  
   → Nhiều file CSV → Gộp thành 1 DataFrame duy nhất (~24k dòng)  
   → Lưu thành `full.csv`

2. **Chuẩn hóa giá tiền & diện tích**  
   → Convert cột `price` → float (triệu VND), xử lý "thương lượng", "triệu", "k", ...  
   → Convert cột `area` → float (m²), xử lý mọi biến thể "m2", "m²", "mét vuông"

3. **Trích xuất thông tin địa chỉ chi tiết**  
   → Từ cột `location` → tách ra 3 cột mới:  
     - `house_number` (số nhà, hẻm số, ...)  
     - `street_name` (tên đường/phố/ấp/...)  
     - `address` (quận/huyện - tỉnh/thành phố, dạng rút gọn)

4. **Trích xuất & mã hóa tiện ích từ mô tả**  
   → Từ cột `thongtinmota` → tạo 10+ cột binary (0/1) đại diện cho các tiện ích:  
     máy lạnh, tủ lạnh, máy giặt, gác lửng, bếp, tủ quần áo, giường, ban công, thang máy, giờ giấc tự do, chỗ để xe

5. **Chuẩn hóa ngày đăng**  
   → Từ cột `ngaydang` → chuyển thành cột `date` dạng datetime (pd.to_datetime)

6. **Lọc cột & đổi tên chuẩn hóa**  
   → Giữ lại các cột quan trọng, đổi tên sang tiếng Anh rõ nghĩa  
   → Loại bỏ cột thừa, chuẩn bị cấu trúc cuối cùng

7. **Lưu kết quả sạch**  
   → Xuất file CSV cuối cùng: `cleaned_dataX.csv` (encoding utf-8-sig để mở Excel không lỗi tiếng Việt)

### **Bảng tóm tắt chi tiết từng bước**

| Thứ tự | Bước xử lý                              | Cột đầu vào chính              | Kỹ thuật chính                              | Cột đầu ra / thay đổi chính                     | Ghi chú quan trọng                              |
|--------|-----------------------------------------|--------------------------------|---------------------------------------------|--------------------------------------------------|-------------------------------------------------|
| 1      | Gộp dữ liệu                             | Tất cả file Page*to*.csv       | glob + pd.concat                            | `full.csv` (~24k dòng)                           | index=False, encoding='utf-8-sig'               |
| 2      | Convert giá tiền                        | price                          | regex + heuristic (triệu, k, thương lượng)  | `price` (float, triệu VND) hoặc NaN              | Ưu tiên số lớn nhất hợp lý                      |
| 3      | Convert diện tích                       | area                           | regex (m², m2, mét vuông)                   | `area` (float, m²) hoặc NaN                      | Lấy số cuối cùng xuất hiện                      |
| 4      | Trích xuất địa chỉ                      | location                       | regex phức tạp + fix unicode + clean prefix | `house_number`, `street_name`, `address`         | Xử lý lỗi ĐưỜng → Đường, ấp, hẻm, ngõ...        |
| 5      | Trích xuất tiện ích                     | thongtinmota                   | normalize + keyword dict + fuzzy matching + negative rule | 10+ cột binary: air_conditioning, fridge, ... | Ưu tiên phát hiện phủ định ("không có...")     |
| 6      | Chuẩn hóa ngày đăng                     | ngaydang                       | regex dd/mm/yyyy + pd.to_datetime           | `date` (datetime)                                | Xử lý các định dạng ngày phổ biến               |
| 7      | Chọn lọc & đổi tên cột                  | —                              | rename + column selection                   | DataFrame gọn gàng, tên tiếng Anh                | Chỉ giữ cột cần thiết cho phân tích             |
| 8      | Lưu file sạch                           | —                              | to_csv(utf-8-sig)                           | `cleaned_data6.csv` (hoặc phiên bản mới)         | Sẵn sàng cho EDA & modeling                     |

### **Các tiện ích được trích xuất (binary 0/1)**

- **air_conditioning** : máy lạnh / điều hòa  
- **fridge**           : tủ lạnh  
- **washing_machine**  : máy giặt  
- **mezzanine**        : gác lửng / gác xép  
- **kitchen**          : có bếp / được nấu ăn  
- **wardrobe**         : tủ quần áo  
- **bed**              : giường / nệm  
- **balcony**          : ban công  
- **elevator**         : thang máy  
- **free_time**        : giờ giấc tự do / ra vào thoải mái  
- **parking**          : chỗ để xe / bãi xe / hầm xe  

### **Cấu trúc file kết quả cuối cùng (các cột)**
```text
title
description              (từ thongtinmota)
location                 (địa chỉ gốc)
address                  (quận/huyện - tỉnh/thành)
street_name
price                    (triệu VND)
area                     (m²)
date                     (datetime)
air_conditioning         (0/1)
fridge                   (0/1)
washing_machine          (0/1)
mezzanine                (0/1)
kitchen                  (0/1)
wardrobe                 (0/1)
bed                      (0/1)
balcony                  (0/1)
elevator                 (0/1)
free_time                (0/1)
parking                  (0/1)
```


## **Thực nghiệm (Experimentation)**
### **Bước 1: Import các thư viện cần thiết**

**Mục đích**: Khai báo tất cả các thư viện sẽ sử dụng trong toàn bộ pipeline để xử lý dữ liệu, regex, chuẩn hóa văn bản, fuzzy matching, v.v.

**Lưu ý**: Sử dụng `fuzzywuzzy` để xử lý lỗi chính tả trong phần trích xuất tiện ích.

In [2]:
import pandas as pd
import numpy as np
import re
import glob
import os
import unicodedata
%pip install fuzzywuzzy
from fuzzywuzzy import fuzz

import sys
import os

# Thêm thư mục cha (thư mục gốc của project) vào hệ thống đường dẫn của Python
sys.path.append(os.path.abspath('..'))

# Bây giờ bạn có thể gọi hàm từ utilities
from utilities.processing import *


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### **Bước 2: Gộp nhiều file CSV thành một DataFrame duy nhất**

**Mục đích**: Kết hợp tất cả các file crawl (Page*to*.csv) thành một bảng dữ liệu thống nhất để xử lý tập trung.

**Input**: Nhiều file CSV trong thư mục `../Data/raw/`  
**Output**: DataFrame `raw` + lưu file `full.csv`  
**Kỹ thuật**: `glob` + `pd.concat`

In [ ]:
input_path = '../Data'
files = glob.glob(os.path.join(input_path, "Page*to*.csv"))

List = []
for filename in files:
    df = pd.read_csv(filename, index_col=None, header=0)
    List.append(df)

raw = pd.concat(List, axis=0, ignore_index=True)

print(f"Tổng số dòng dữ liệu: {raw.shape[0]}")
raw.head()

output_path = '../Data/raw.csv'
raw.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"Đã lưu thành công file gộp tại: {output_path}")


Tổng số dòng dữ liệu: 24121
Đã lưu thành công file gộp tại: ../Data/full_raw.csv


### **Bước 3: Chuẩn hóa & chuyển đổi cột giá tiền và diện tích**

**Mục đích**: Đưa giá tiền về đơn vị **triệu VND** (float) và diện tích về **m²** (float), xử lý các biến thể văn bản phổ biến ở Việt Nam.

**Input**: cột `price`, `area` (string)  
**Output**: cột `price` và `area` dạng số (float), giá trị không hợp lệ → NaN  
**Kỹ thuật**: regex + heuristic logic (triệu/tr, k, thương lượng, m²/m2/mét vuông...)

In [4]:

raw['price'] = raw['price'].apply(covert_price)
raw['area'] = raw['area'].apply(covert_area)


### **Bước 4: Trích xuất thông tin địa chỉ chi tiết**

**Mục đích**: Từ cột `location` (địa chỉ đầy đủ) tách ra:
- `house_number`: số nhà / số hẻm
- `street_name`: tên đường/phố/ấp/...
- `address`: phần quận/huyện - tỉnh/thành phố (rút gọn)

**Kỹ thuật**: regex phức tạp + fix unicode + clean prefix + fallback logic

In [5]:
raw.rename(columns={'address': 'location'}, inplace=True)

raw[['house_number', 'street_name']] = raw['location'].apply(extract_street)
raw['address'] = raw['location'].apply(extract_address)


### **Bước 5: Trích xuất & mã hóa các tiện ích từ mô tả**

**Mục đích**: Phân tích nội dung cột `thongtinmota` để tạo các cột binary (0/1) biểu thị sự hiện diện của từng tiện ích phổ biến.

**Input**: cột `thongtinmota`  
**Output**: 11 cột binary mới (air_conditioning, fridge, washing_machine, ...)  
**Kỹ thuật**: normalize text + từ điển keyword + negative rule + fuzzy matching

In [6]:
amenity_keywords = {
    'ac': {'positive': ['máy lạnh', 'điều hòa', 'có máy lạnh'], 'negative': ['không có máy lạnh'], 'fuzzy': 90},
    'fridge': {'positive': ['tủ lạnh'], 'negative': ['không tủ lạnh'], 'fuzzy': 90},
    'washing_machine': {'positive': ['máy giặt'], 'negative': ['không máy giặt'], 'fuzzy': 85},
    'mezzanine': {'positive': ['gác lửng', 'gác cao'], 'negative': ['không gác'], 'fuzzy': 90},
    'kitchen': {'positive': ['bếp', 'nấu ăn', 'được nấu'], 'negative': ['không nấu ăn'], 'fuzzy': 85},
    'wardrobe': {'positive': ['tủ quần áo'], 'negative': [], 'fuzzy': 90},
    'bed': {'positive': ['giường', 'nệm'], 'negative': [], 'fuzzy': 90},
    'balcony': {'positive': ['ban công'], 'negative': [], 'fuzzy': 90},
    'elevator': {'positive': ['thang máy'], 'negative': [], 'fuzzy': 90},
    'free_time': {'positive': ['giờ giấc tự do', 'ra vào tự do'], 'negative': ['giờ giới nghiêm'], 'fuzzy': 85},
    'parking': {'positive': ['bãi xe', 'hầm xe'], 'negative': ['không để xe'], 'fuzzy': 85}
}

for col_name, config in amenity_keywords.items():
    raw[col_name] = raw['thongtinmota'].apply(lambda x: amenity(x, config))


### **Bước 6: Chuẩn hóa cột ngày đăng**

**Mục đích**: Chuyển đổi ngày đăng về định dạng datetime chuẩn để dễ phân tích thời gian (ví dụ: lọc theo tháng, tính độ mới của tin đăng).

**Input**: cột `ngaydang`  
**Output**: cột `date` (datetime)  
**Kỹ thuật**: regex tìm dd/mm/yyyy + `pd.to_datetime`

In [7]:
raw['date'] = raw['ngaydang'].apply(process_date)


### **Bước 7: Lọc cột cần thiết, đổi tên & lưu kết quả cuối cùng**

**Mục đích**: Giữ lại các cột quan trọng, đổi tên sang tiếng Anh chuẩn, xuất file sạch để sử dụng tiếp theo.

**Output**: file `cleaned_data6.csv` (hoặc tên mới)

In [8]:
columns = [
    'title', 'thongtinmota', 'location', 'address', 'street_name', 'price', 'area',
    'date', 'ac', 'fridge', 'washing_machine', 'mezzanine', 'kitchen', 'wardrobe',
    'bed', 'balcony', 'elevator', 'free_time', 'parking', 'url'
]

cleaned = raw[columns].copy()
cleaned.rename(columns={'ac': 'air_conditioning', 'thongtinmota': 'description'}, inplace=True)

output_path = '../Data/cleaned.csv'
cleaned.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"Đã lưu dữ liệu sạch vào: {output_path}")
print(cleaned.info())


Đã lưu dữ liệu sạch vào: ../Data/cleaned.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24121 entries, 0 to 24120
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             24121 non-null  object        
 1   description       24113 non-null  object        
 2   location          24113 non-null  object        
 3   address           24121 non-null  object        
 4   street_name       24121 non-null  object        
 5   price             24062 non-null  float64       
 6   area              24113 non-null  float64       
 7   date              24113 non-null  datetime64[ns]
 8   air_conditioning  24121 non-null  int64         
 9   fridge            24121 non-null  int64         
 10  washing_machine   24121 non-null  int64         
 11  mezzanine         24121 non-null  int64         
 12  kitchen           24121 non-null  int64         
 13  wardrobe          24121 non-nul

### **<u>Lưu ý:</u> Đề xuất cải tiến cho các bước tiếp theo**

- Loại bỏ outlier (giá < 0.5tr hoặc > 100tr, diện tích < 8m² hoặc > 500m²)  
- Phân loại loại hình nhà/phòng từ title + description (phòng trọ, căn hộ, nhà nguyên căn, mặt bằng...)  
- Mapping chuẩn tên quận/huyện/thành phố (tạo bảng tra cứu)  
- Tính thêm cột `price_per_m2`  
- Thay rule-based bằng mô hình ngôn ngữ nhỏ (PhoBERT/viBERT) để trích xuất tiện ích chính xác hơn